In [1]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging
import torch
import pandas as pd
import pickle as pk
import os
import random
import json
import time, datetime
random.seed(64)
import re

from helpers import *
from keep_records import Keep_records

print(f"Cuda: {torch.cuda.is_available()}")

Cuda: True


In [2]:
# Load the json to find the dice pips
# Load the text files into a df
datafolder = "data/norec/" # 'data' folder from norec github repo
with open(datafolder+"metadata.json") as rf:
    metadata = json.load(rf)
df_splits = {}
for split in ['train', 'dev', 'test']:
    splitfolder = datafolder+split
    filenames = os.listdir(splitfolder)
    texts, scores = [], []
    for filename in filenames:
        with open (os.path.join(splitfolder, filename)) as rf:
            texts.append(rf.read().strip())
        scores.append(float(metadata[filename.split(".")[0]]['rating'])/6) #rating up tp 6
    df_splits[split] = pd.DataFrame({'texts': texts, 'labels': scores})

In [3]:
[print(k, len(v)) for k,v in df_splits.items()]

train 34903
dev 4360
test 4351


[None, None, None]

In [4]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

model_args = ClassificationArgs(sliding_window=True)
model_args.num_train_epochs = 1
model_args.regression = True
model_args.overwrite_output_dir = True
model_args.silent = True
model = ClassificationModel(
    "bert",
    "bert-base-multilingual-cased",
    num_labels=1,
    args=model_args
)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [5]:
out_d = "outputs/simpletransformers/norec/"
model.train_model(df_splits['train'],
        output_dir = out_d)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled
INFO:simpletransformers.classification.classification_model: 248500 features created from 34903 samples.
INFO:simpletransformers.classification.classification_model: Training of bert model complete. Saved to outputs/simpletransformers/norec/.


In [6]:
result, model_outputs, wrong_predictions = model.eval_model(df_splits['dev'])

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled
INFO:simpletransformers.classification.classification_model: 4360 features created from 4360 samples.
INFO:simpletransformers.classification.classification_model:{'eval_loss': 0.025358992738838636}


In [43]:
from collections import Counter
Counter(scores)

Counter({0.8333333333333334: 1714,
         0.5: 707,
         0.6666666666666666: 1390,
         0.3333333333333333: 243,
         1.0: 270,
         0.16666666666666666: 27})

In [36]:

predictions, raw_outputs = model.predict([df_splits["dev"]["texts"].iloc[3]])
print(predictions)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled
INFO:simpletransformers.classification.classification_model: 1 features created from 1 samples.
[0]


In [34]:
df_splits["dev"]["texts"].iloc[2]


"Wayward Pines S01 E01-05\nStarter med et smell, men det gode utgangspunktet smuldrer raskt bort. \n\nNavnet Wayward Pines antyder, i likhet med Twin Peaks, en liten og mystisk amerikansk småby. \nDen sistnevnte kultserien har vært en åpenbar inspirasjon for produsent M. Night Shyamalan og de andre serieskaperne i arbeidet med førstnevnte. \n\nI sentrum for hendelsene i Wayward Pines er Matt Dillon i rollen som NSA-agenten Ethan Burke. \nHan er sendt ut på hemmelig oppdrag for å lokalisere to savnede agenter. \nEn brutral og plutselig kollisjon på reisen sender ham på sykehus, hvor han våkner alene og forvirret. \n\nEn lege insisterer på å utføre en hjerneoperasjon, men Burke kjemper seg fri og finner utenfor døren et samfunn hvor ingenting er slik det først utgir seg for å være; byen Wayward Pines. \n\nKopierer andre i stedet for å finne sin egenart \n\nInspirasjonskilden Twin Peaks er også en kjepphest – Wayward Pines er en serie som ikke etablerer en egen troverdig ramme rundt sine 

## History
First attempt
Tried to do regression, but got only 0 out.
```
model_args = ClassificationArgs(sliding_window=True)
model_args.num_train_epochs = 1
model_args.regression = True
model_args.overwrite_output_dir = True
model_args.silent = True
model = ClassificationModel(
    "bert",
    "bert-base-multilingual-cased",
    num_labels=1,
    args=model_args
)
```

